In [7]:
import os
import shutil
import numpy as np
from PIL import Image
from skimage import data
from sklearn.cluster import KMeans
import pprint

In [6]:
# 1. 国旗画像のサイズをそろえて保存する
# ./flag_origin 以下に国旗画像
# ./flag_convert 以下に200*100のサイズに変換したjpgを保存
for path in os.listdir('./flag_origin'):
    img = Image.open(f'./flag_origin/{path}')
    img = img.convert('RGB')
    img_resize = img.resize((200, 100))
    img_resize.save(f'./flag_convert/{path}.jpg')

In [22]:
aa = os.listdir('./flag_convert')
pprint.pprint(aa)

['007.png.jpg',
 '010.png.jpg',
 '013.png.jpg',
 '008.png.jpg',
 '012.png.jpg',
 '011.png.jpg',
 '005.png.jpg',
 '014a.png.jpg',
 '014.png.jpg',
 '006.png.jpg',
 '009.png.jpg']


In [9]:
# 2. 3次元配列の画像データを2次元配列のデータに変換
feature = np.array([data.imread(f'./flag_convert/{path}') for path in os.listdir('./flag_convert')])
print(feature.shape)
pprint.pprint(feature)

(11, 100, 200, 3)
array([[[[255, 248, 255],
         [255, 252, 255],
         [233, 233, 245],
         ...,
         [  0,  38, 119],
         [  0,  38, 119],
         [  0,  38, 119]],

        [[255, 245, 251],
         [255, 252, 255],
         [255, 250, 255],
         ...,
         [  0,  38, 119],
         [  0,  38, 119],
         [  0,  38, 119]],

        [[219, 204, 201],
         [219, 204, 201],
         [231, 215, 216],
         ...,
         [  0,  38, 119],
         [  0,  38, 119],
         [  0,  38, 119]],

        ...,

        [[  0,  38, 119],
         [  0,  38, 119],
         [  0,  38, 119],
         ...,
         [  0,  38, 119],
         [  0,  38, 119],
         [  0,  38, 119]],

        [[  0,  38, 119],
         [  0,  38, 119],
         [  0,  38, 119],
         ...,
         [  0,  38, 119],
         [  0,  38, 119],
         [  0,  38, 119]],

        [[  0,  38, 119],
         [  0,  38, 119],
         [  0,  38, 119],
         ...,
         [  0,  

In [13]:
feature = feature.reshape(len(feature), -1).astype(np.float64)
print(feature.shape)
print(feature[0][0])
print(feature[0][1])
print(feature[0][2])
print(feature[0][3])
print(feature[0][4])
pprint.pprint(feature)

(11, 60000)
255.0
248.0
255.0
255.0
252.0
array([[255., 248., 255., ...,   0.,  38., 119.],
       [255.,   0.,  32., ..., 255.,   0.,  32.],
       [106., 173., 218., ..., 106., 173., 218.],
       ...,
       [  0.,  58., 183., ..., 196., 196., 196.],
       [239.,  26.,  32., ...,   0.,   5., 133.],
       [  0.,  37., 144., ..., 245.,  23.,  64.]])


In [15]:
# 3. 学習(15種類のグループにクラスタリングする)
model = KMeans(n_clusters=3).fit(feature)

In [29]:
print(type(model))
pprint.pprint(model)
centers = model.cluster_centers_
print(type(centers))
pprint.pprint(centers)
print(len(centers))
print(len(centers[0]))

<class 'sklearn.cluster.k_means_.KMeans'>
KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=3, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)
<class 'numpy.ndarray'>
array([[164.66666667, 103.66666667, 143.66666667, ...,   0.        ,
         26.66666667, 132.        ],
       [146.2       ,  36.8       ,  65.6       , ..., 237.4       ,
         11.2       ,  46.        ],
       [ 35.33333333,  96.33333333, 194.66666667, ..., 166.        ,
        188.33333333, 203.33333333]])
3
60000


In [ ]:
# 上記の中心点[60000次元]と各画像の点[60000次元]のユークリッド距離をとる
# 距離が最も小さい画像が、中心となる画像とする

In [19]:
# 4. 学習結果のラベル
labels = model.labels_
print(type(labels))
pprint.pprint(labels)

<class 'numpy.ndarray'>
array([0, 1, 2, 1, 1, 1, 0, 2, 2, 0, 1], dtype=int32)


In [20]:
# 5. 学習結果(クラスタリング結果の表示 + ラベルごとにフォルダ分け)
# ./flag_group 以下に画像を分けて保存する
for label, path in zip(labels, os.listdir('./flag_convert')):
    os.makedirs(f"./flag_group/{label}", exist_ok=True)
    shutil.copyfile(f"./flag_origin/{path.replace('.jpg', '')}", f"./flag_group/{label}/{path.replace('.jpg', '')}")
    print(label, path)

0 007.png.jpg
1 010.png.jpg
2 013.png.jpg
1 008.png.jpg
1 012.png.jpg
1 011.png.jpg
0 005.png.jpg
2 014a.png.jpg
2 014.png.jpg
0 006.png.jpg
1 009.png.jpg
